# Exercise 5

## Group 
- **ID**: <your group ID>

- **Members**: 
    - <your name1>
    - <your name2>
    - <your name3>

## Hand-in
- Please hand in this notebook with your code implementation via Ilias 
- Please make sure that there is exactly **one** submission per group

## Task Description

In this exercise, you will implement the DPO algorithm and integrate it into a finetuning pipeline. The algorithm is described in the paper [DPO: Direct Preference Optimization for Language Models](https://arxiv.org/abs/2305.18290).
You will use DPO to finetune a pretrained language model which is already aligned to conversational tasks.

You will use the dataset published by Anthropic. The dataset is a collection of human preference comparisons between two model outputs. The dataset is available at [Huggingface (Anthropic's dataset)](https://huggingface.co/datasets/Anthropic/hh-rlhf).

## Grading scheme
Total: 5 points
1. **Preparing the Dataloader** (1 point)
2. **Loss Function** (1.5 points)
3. **Choose a suitable Model** (0.5 points)
4. **Training loop** (1.5 points)
5. **Generation of Question Answer pairs** (0.5 points)

1. ## Preparing the Dataloader (1 point) <br>
Get familiar with the dataset and prepare the dataloader.

2. ## Loss Function (1.5 points) <br>
Implement the DPO loss function.

3. ## Choose a suitable Model (0.5 points) <br>
Choose a **suitable** model **for the DPO finetuning**. You can use any pretrained model from the Huggingface Hub. Please justify your choice in 1-2 sentences.

In [ ]:
model_name = "___YOUR_MODEL_NAME___"  # Replace with your model name

4. ## Training loop (1.5 points) <br>
Implement the training loop. Which uses your dataloader, the DPO loss function and the model you chose. Also use sensible hyperparameters.

5. ## Generation of Question Answer pairs (0.5 points) <br>
Generate question-answer pairs using the finetuned model and compare them to the base model.